In [ ]:
!pip install pyspark

In [ ]:
#Pour lire un fichier CSV et le charger dans un DataFrame Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL Students").getOrCreate()

# Charger le fichier CSV
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/etudiants.csv", header=True, inferSchema=True)
df.show()

+---------+-------+---+------------+
|      nom| prenom|age|     filiere|
+---------+-------+---+------------+
|Benkirane|  Sarah| 21|Informatique|
| ElAmrani|  Ahmed| 19|     Gestion|
|  Hamdani|  Salma| 22|Informatique|
|     Tazi|   Omar| 20|   Marketing|
|   Fadili|Yasmine| 23|Data Science|
|ElKhayati|    Ali| 18|Informatique|
| Moujahid|  Leila| 25|Data Science|
|  Chraibi| Hassan| 24|   Marketing|
| Boukhari|  Imane| 22|     Gestion|
| Zerouali| Rachid| 21|Informatique|
| Mahmoudi|  Amina| 20|Data Science|
|  Dahmani|  Karim| 23|     Gestion|
|  Idrissi|  Noura| 19|   Marketing|
|  Essaidi|  Samir| 26|Data Science|
+---------+-------+---+------------+



In [ ]:
from pyspark.sql.functions import upper, col  # Import upper and col functions

# Convertir les noms en majuscules
df1 = df.withColumn("nom", upper(col("nom")))
df1.show()

+---------+-------+---+------------+
|      nom| prenom|age|     filiere|
+---------+-------+---+------------+
|BENKIRANE|  Sarah| 21|Informatique|
| ELAMRANI|  Ahmed| 19|     Gestion|
|  HAMDANI|  Salma| 22|Informatique|
|     TAZI|   Omar| 20|   Marketing|
|   FADILI|Yasmine| 23|Data Science|
|ELKHAYATI|    Ali| 18|Informatique|
| MOUJAHID|  Leila| 25|Data Science|
|  CHRAIBI| Hassan| 24|   Marketing|
| BOUKHARI|  Imane| 22|     Gestion|
| ZEROUALI| Rachid| 21|Informatique|
| MAHMOUDI|  Amina| 20|Data Science|
|  DAHMANI|  Karim| 23|     Gestion|
|  IDRISSI|  Noura| 19|   Marketing|
|  ESSAIDI|  Samir| 26|Data Science|
+---------+-------+---+------------+



In [ ]:
#Filtrer les étudiants de plus de 20 ans
df2= df1.filter(df1["age"] > 20)
df2.show()

+---------+-------+---+------------+
|      nom| prenom|age|     filiere|
+---------+-------+---+------------+
|BENKIRANE|  Sarah| 21|Informatique|
|  HAMDANI|  Salma| 22|Informatique|
|   FADILI|Yasmine| 23|Data Science|
| MOUJAHID|  Leila| 25|Data Science|
|  CHRAIBI| Hassan| 24|   Marketing|
| BOUKHARI|  Imane| 22|     Gestion|
| ZEROUALI| Rachid| 21|Informatique|
|  DAHMANI|  Karim| 23|     Gestion|
|  ESSAIDI|  Samir| 26|Data Science|
+---------+-------+---+------------+



In [ ]:
#Calculer la moyenne d'âge par filière
from pyspark.sql.functions import avg
stats = df2.groupBy("Filiere").agg(avg("age").alias("moyenne_age"))
stats.show()

NameError: name 'df2' is not defined

In [ ]:
#Sauvegarde les données transformées dans un fichier CSV
df2.write.csv("/content/etudiants_transforme.csv", header=True)

In [ ]:
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 14.2 MB/s eta 0:00:00


In [ ]:
import csv
import json
from kafka import KafkaProducer

# Configuration du producteur Kafka
producer = KafkaProducer(
    bootstrap_servers='kafka-cda26e1-ibenchayb-eaa2.h.aivencloud.com:10874',
    security_protocol='SSL',
    ssl_cafile='/content/drive/MyDrive/Colab Notebooks/Exam_BI_Moha/ca.pem',
    ssl_certfile='/content/drive/MyDrive/Colab Notebooks/Exam_BI_Moha/service.cert',
    ssl_keyfile='/content/drive/MyDrive/Colab Notebooks/Exam_BI_Moha/service.key',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # Convertir les données en JSON
)

# Chemin du fichier CSV
csv_file_path = "/content/drive/MyDrive/Colab Notebooks/Exam_BI_Moha/etudiants.csv"

# Lire le fichier CSV et envoyer chaque ligne à Kafka
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)  # Lire les lignes sous forme de dictionnaire
    for row in csv_reader:
        producer.send('mohamed', value=row)  # Envoyer la ligne comme message JSON
        print(f"Message envoyé : {row}")

# Finaliser l'envoi
producer.flush()
print("Tous les messages ont été envoyés !")

Message envoyé : {'nom': 'Benkirane', 'prenom': 'Sarah', 'age': '21', 'filiere': 'Informatique'}
Message envoyé : {'nom': 'ElAmrani', 'prenom': 'Ahmed', 'age': '19', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Hamdani', 'prenom': 'Salma', 'age': '22', 'filiere': 'Informatique'}
Message envoyé : {'nom': 'Tazi', 'prenom': 'Omar', 'age': '20', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Fadili', 'prenom': 'Yasmine', 'age': '23', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'ElKhayati', 'prenom': 'Ali', 'age': '18', 'filiere': 'Informatique'}
Message envoyé : {'nom': 'Moujahid', 'prenom': 'Leila', 'age': '25', 'filiere': 'Data Science'}
Message envoyé : {'nom': 'Chraibi', 'prenom': 'Hassan', 'age': '24', 'filiere': 'Marketing'}
Message envoyé : {'nom': 'Boukhari', 'prenom': 'Imane', 'age': '22', 'filiere': 'Gestion'}
Message envoyé : {'nom': 'Zerouali', 'prenom': 'Rachid', 'age': '21', 'filiere': 'Informatique'}
Message envoyé : {'nom': 'Mahmoudi', 'prenom': 'Amina', 'age'

In [ ]:
from kafka import KafkaConsumer
import json

# Configuration du consommateur Kafka
TOPIC_NAME = "mohamed"
consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers='kafka-cda26e1-ibenchayb-eaa2.h.aivencloud.com:10874',
    client_id="CONSUMER_CLIENT_ID",
    group_id="CONSUMER_GROUP_ID",
    security_protocol='SSL',
    ssl_cafile='/content/drive/MyDrive/Colab Notebooks/Exam_BI_Moha/ca.pem',
    ssl_certfile='/content/drive/MyDrive/Colab Notebooks/Exam_BI_Moha/service.cert',
    ssl_keyfile='/content/drive/MyDrive/Colab Notebooks/Exam_BI_Moha/service.key',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))  # Désérialisation des messages JSON
)

# Consommer les messages et les afficher
print("En attente de messages...")

for message in consumer:
    # Décodage et affichage du message
    print(f"Message reçu : {message.value}")
    # Vous pouvez ajouter une logique supplémentaire pour traiter les données du message ici

En attente de messages...
